In [ ]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.3 MB/s eta 0:00:00


In [3]:
import os
import torch
import torchmetrics
import torch.nn.functional as F

from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from PIL import Image

In [4]:
!gdown --id 1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
!unzip -q dogs-vs-cats.zip
!unzip -q train.zip
!unzip -q test1.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
From (redirected): https://drive.google.com/uc?id=1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk&confirm=t&uuid=1d366134-20e2-4ecb-b6df-9fd28a4e58dd
To: /content/dogs-vs-cats.zip
100% 852M/852M [00:12<00:00, 66.7MB/s]


In [5]:
class Dataset():
  def __init__(self, filelist, filepath, transform=None):
    self.filelist = filelist
    self.filepath = filepath
    self.transform = transform

  def __len__(self):
    return int(len(self.filelist))

  def __getitem__(self, index):
    imgpath = os.path.join(self.filepath, self.filelist[index])
    img = Image.open(imgpath)

    if "dog" in imgpath:
      label = 1
    else:
      label = 0

    if self.transform is not None:
      img = self.transform(img)

    return (img, label)

In [7]:
train_dir = "./train"
test_dir = "./test1"

train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

transformations = transforms.Compose([transforms.Resize((60, 60)), transforms.ToTensor()])

train = Dataset(train_files, train_dir, transformations)
val = Dataset(test_files, test_dir, transformations)

train, val = torch.utils.data.random_split(train, [20000, 5000])

#train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, shuffle=True)
#val_loader = torch.utils.data.DataLoader(dataset = val, batch_size = 32, shuffle=False)